# Order parameters 
## Parity order parameter 

\begin{equation} 
\mathscr{O}_P = \lim_{|i-j| \to \infty} \langle  e^{i\theta \sum_{i \le k < j} \delta \hat{n}_k}\rangle
\end{equation}

## String Order Parameter 
\begin{equation} 
\mathscr{O}_S = \lim_{|i-j| \to \infty} \langle \delta \hat{n}_i e^{i\theta \sum_{i \le k < j} \delta \hat{n}_k} \delta \hat{n}_j \rangle
\end{equation}

In [39]:
using LinearAlgebra 
using Plots #Graph
using SparseArrays #Sparse matrices 
using LaTeXStrings #Titles and labels 
using JLD #Save data 
#definition of N,M,D
global N=7
global M=7
global D=Int((factorial(M+N-1))/(factorial(M-1)*factorial(N)))  

#i-prime number function(suggested form) 
p(i)=100i+3 

#Generate a basis for N = M
function generate_basis(N, M)
    D = prod(max(N, M):N+M-1) ÷ prod(1:min(N, M))
    basis = [zeros(Int, M) for _ in 1:D]
    basis[1][1] = N
    for t = 2:D
        if basis[t-1][M] != 0
            k = M - 1
        else
            k = M
        end
        while k > 0 && basis[t-1][k] == 0
            k -= 1
        end
        @views basis[t][1:k-1] .= basis[t-1][1:k-1]
        basis[t][k] = basis[t-1][k] - 1
        basis[t][k+1] = N - sum(@view(basis[t][1:k])) 
        end 
    return basis  
end

generate_basis (generic function with 1 method)

In [40]:
v=generate_basis(N,M);

In [41]:
function occupation(i, v) 
     if(v[i] >= 1)  
        return  v[i]  
    else 
        return 0;
    end  
end 

occupation (generic function with 1 method)

In [42]:
function matrixoccupation(k, D) 
    mat = spzeros(D,D)
    for j in 1:D 
        mat[j,j] = occupation(k, v[j]) 
    end   
    return mat 
end

matrixoccupation (generic function with 1 method)

In [9]:
Matrix(sum(matrixoccupation(i,D) for i in 1:(N-1))) - (N-1)*I == matrixoccupation(1, D) + matrixoccupation(2, D)  - 2*I #matrixoccupation(4, D) - 4*I

true

In [15]:
exp(1im*pi*Matrix((matrixoccupation(1, D) + matrixoccupation(2, D) - 2*I ))) #matrixoccupation(3, D) + matrixoccupation(4, D) - 4*I)))

0.9999999999999998 + 4.702549984389891e-16im

In [8]:
exp(1im*pi)

-1.0 + 1.2246467991473532e-16im

In [9]:
#sum( matrixoccupation(i,D) for i in 2:(N-1) ) == matrixoccupation(2,D)

In [43]:
#Parity order parameter 
theta = pi
Op = exp(Matrix(sum(1im*theta*matrixoccupation(i,D) for i in 1:(N-1))) - 1im*theta*(N-1)*I)
Op = real(Op);

In [33]:
#Ops == Opx

In [44]:
#Op

In [45]:
#String Order parameter (we fix the parameters i =1, j = N, and i<=k<j ) 
Ots = (exp(Matrix(sum(1im*theta*matrixoccupation(i,D) for i in 1:(N-1))) - 1im*theta*(N-1)*I))*(Matrix(matrixoccupation(N,D)) - I)
Os = (Matrix(matrixoccupation(1,D)) - I)*Ots 
Os = real(Os);

In [46]:
#Os

In [30]:
#Optim Example for 
#f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2  
#x0 = [0.0, 0.0]  
#optimize(f, x0)  
# optimize(f, zeros(2) )  
##optimize(x->dot(x, Os*x), zeros(D)) ###NO CONSTRAINTS! so we will use JuMP package 
using JuMP
import Ipopt

In [21]:
function ParityOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, x[1:D] >= 0)
    @objective(model, Min, z)  
    @constraint(model, dot(x, Op*x) == z)
    @constraint(model, sum(x.^2) == 1)
    optimize!(model)
    if verbose
        print(model)
        println("Objective value: ", objective_value(model))
        println("Op = ", value(z))
        for i in 1:D 
            println("x_$i = ", value(x[i]) ) 
        end
    end
    return
end

#ParityOrder()

ParityOrder (generic function with 1 method)

In [15]:
function StringOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, 0 <= x[1:D] <= 1 )
    @objective(model, Max, z)   
    @constraint(model, dot(x,Op*x) == 0)
    @constraint(model, dot(x, Os*x) == z)
    @constraint(model, sum(x.^2) == 1)
    optimize!(model)
    if verbose
        print(model)
        println("Objective value: ", objective_value(model))
        println("Os = ", value(z))
        for i in 1:D 
            println("x_$i = ", value(x[i]) ) 
        end
    end
    return
end

StringOrder() 
#println("square root of the probability amplitude ")

Objective value: 0.07326255558121757
Os = 0.07326255558121757
x_1 = 8.857481421899604e-7
x_2 = 8.856965003243175e-7
x_3 = 8.856965003243175e-7
x_4 = 8.856965003243175e-7
x_5 = 1.5122385989454973e-6
x_6 = 8.857906275359124e-7
x_7 = 8.857906275359124e-7
x_8 = 8.857906275359124e-7
x_9 = 1.5122385989454973e-6
x_10 = 8.857906275359124e-7
x_11 = 8.857906275359124e-7
x_12 = 1.5122385989454973e-6
x_13 = 8.857906275359124e-7
x_14 = 1.5122385989454973e-6
x_15 = 2.56476218672957e-6
x_16 = 8.858112918962811e-7
x_17 = 8.858112918962811e-7
x_18 = 8.858112918962811e-7
x_19 = 1.5122385989454973e-6
x_20 = 8.858112918962811e-7
x_21 = 8.858112918962811e-7
x_22 = 1.5122385989454973e-6
x_23 = 8.858112918962811e-7
x_24 = 1.5122385989454973e-6
x_25 = 2.5639212854121867e-6
x_26 = 8.858112918962811e-7
x_27 = 8.858112918962811e-7
x_28 = 1.5122385989454973e-6
x_29 = 8.858112918962811e-7
x_30 = 1.5122385989454973e-6
x_31 = 2.5639212854121867e-6
x_32 = 8.858112918962811e-7
x_33 = 1.5122385989454973e-6
x_34 = 2.563

In [47]:
#Until N
coefficients = Vector{Float64}()
function StringOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, 0 <= x[1:D] <= 1 )
    @objective(model, Max, z)   
    @constraint(model, sum(Op*x) == 0)
    @constraint(model, sum(Os*x) == z)
    @constraint(model, sum(x) == 1)
    optimize!(model)
    if verbose
        #print(model)
        #println("Objective value: ", objective_value(model)) 
        #L"Objective~value:"
        println("Os = ", value(z))
        for i in 1:D 
            println("x_$i = ", value(x[i]) )  
            push!(coefficients, value(x[i]))
        end
    end
    return
end

StringOrder()
DictOfBasis = Dict(coefficients .=> v) 
println("The largest probability amplitude is ", maximum(coefficients) ," corresponding to the basis vector " , 
DictOfBasis[maximum(coefficients)] )

Os = 6.000100031818417
x_1 = 0.5000108560191395
x_2 = -9.090909130962423e-9
x_3 = -9.090909130962423e-9
x_4 = -9.090909130962423e-9
x_5 = -9.090909130962423e-9
x_6 = -9.090909130962423e-9
x_7 = -9.848484849058364e-9
x_8 = -9.545454555610285e-9
x_9 = -9.545454555610285e-9
x_10 = -9.545454555610285e-9
x_11 = -9.545454555610285e-9
x_12 = -9.545454555610285e-9
x_13 = -9.848484849058364e-9
x_14 = -9.545454555610285e-9
x_15 = -9.545454555610285e-9
x_16 = -9.545454555610285e-9
x_17 = -9.545454555610285e-9
x_18 = -9.848484849058364e-9
x_19 = -9.545454555610285e-9
x_20 = -9.545454555610285e-9
x_21 = -9.545454555610285e-9
x_22 = -9.848484849058364e-9
x_23 = -9.545454555610285e-9
x_24 = -9.545454555610285e-9
x_25 = -9.848484849058364e-9
x_26 = -9.545454555610285e-9
x_27 = -9.848484849058364e-9
x_28 = -9.909090909501616e-9
x_29 = -9.69696970150418e-9
x_30 = -9.69696970150418e-9
x_31 = -9.69696970150418e-9
x_32 = -9.69696970150418e-9
x_33 = -9.69696970150418e-9
x_34 = -9.848484849058364e-9
x_35 = -

x_293 = -9.818181819820193e-9
x_294 = -9.848484849058364e-9
x_295 = -9.818181819820193e-9
x_296 = -9.848484849058364e-9
x_297 = -9.870129870967074e-9
x_298 = -9.818181819820193e-9
x_299 = -9.848484849058364e-9
x_300 = -9.870129870967074e-9
x_301 = -9.77272727401708e-9
x_302 = -9.818181819820193e-9
x_303 = -9.818181819820193e-9
x_304 = -9.818181819820193e-9
x_305 = -9.848484849058364e-9
x_306 = -9.818181819820193e-9
x_307 = -9.818181819820193e-9
x_308 = -9.848484849058364e-9
x_309 = -9.818181819820193e-9
x_310 = -9.848484849058364e-9
x_311 = -9.870129870967074e-9
x_312 = -9.818181819820193e-9
x_313 = -9.818181819820193e-9
x_314 = -9.848484849058364e-9
x_315 = -9.818181819820193e-9
x_316 = -9.848484849058364e-9
x_317 = -9.870129870967074e-9
x_318 = -9.818181819820193e-9
x_319 = -9.848484849058364e-9
x_320 = -9.870129870967074e-9
x_321 = -9.77272727401708e-9
x_322 = -9.818181819820193e-9
x_323 = -9.818181819820193e-9
x_324 = -9.848484849058364e-9
x_325 = -9.818181819820193e-9
x_326 = -9.8

x_639 = -9.84848484962364e-9
x_640 = -9.84848484962364e-9
x_641 = -9.848484849058364e-9
x_642 = -9.84848484962364e-9
x_643 = -9.848484849058364e-9
x_644 = -9.84848484962364e-9
x_645 = -9.84848484962364e-9
x_646 = -9.848484849058364e-9
x_647 = -9.84848484962364e-9
x_648 = -9.848484849058364e-9
x_649 = -9.84848484962364e-9
x_650 = -9.84848484962364e-9
x_651 = -9.848484849058364e-9
x_652 = -9.84848484962364e-9
x_653 = -9.848484849058364e-9
x_654 = -9.84848484962364e-9
x_655 = -9.848484849058364e-9
x_656 = -9.84848484962364e-9
x_657 = -9.848484849058364e-9
x_658 = -9.84848484962364e-9
x_659 = -9.84848484962364e-9
x_660 = -9.84848484962364e-9
x_661 = -9.84848484962364e-9
x_662 = -9.848484849058364e-9
x_663 = -9.84848484962364e-9
x_664 = -9.84848484962364e-9
x_665 = -9.848484849058364e-9
x_666 = -9.84848484962364e-9
x_667 = -9.848484849058364e-9
x_668 = -9.84848484962364e-9
x_669 = -9.84848484962364e-9
x_670 = -9.84848484962364e-9
x_671 = -9.848484849058364e-9
x_672 = -9.84848484962364e-9
x_

x_986 = -9.870129870967074e-9
x_987 = -9.870129870967074e-9
x_988 = -9.870129870967074e-9
x_989 = -9.848484849058364e-9
x_990 = -9.870129870967074e-9
x_991 = -9.870129870967074e-9
x_992 = -9.848484849058364e-9
x_993 = -9.870129870967074e-9
x_994 = -9.848484849058364e-9
x_995 = -9.818181819820193e-9
x_996 = -9.870129870967074e-9
x_997 = -9.870129870967074e-9
x_998 = -9.870129870967074e-9
x_999 = -9.848484849058364e-9
x_1000 = -9.870129870967074e-9
x_1001 = -9.870129870967074e-9
x_1002 = -9.848484849058364e-9
x_1003 = -9.870129870967074e-9
x_1004 = -9.848484849058364e-9
x_1005 = -9.818181819820193e-9
x_1006 = -9.870129870967074e-9
x_1007 = -9.870129870967074e-9
x_1008 = -9.848484849058364e-9
x_1009 = -9.870129870967074e-9
x_1010 = -9.848484849058364e-9
x_1011 = -9.818181819820193e-9
x_1012 = -9.870129870967074e-9
x_1013 = -9.848484849058364e-9
x_1014 = -9.818181819820193e-9
x_1015 = -9.886363636686314e-9
x_1016 = -9.870129870967074e-9
x_1017 = -9.870129870967074e-9
x_1018 = -9.8701298709

x_1318 = -9.818181819820193e-9
x_1319 = -9.870129870967074e-9
x_1320 = -9.848484849058364e-9
x_1321 = -9.818181819820193e-9
x_1322 = -9.886363636686314e-9
x_1323 = -9.870129870967074e-9
x_1324 = -9.870129870967074e-9
x_1325 = -9.848484849058364e-9
x_1326 = -9.870129870967074e-9
x_1327 = -9.848484849058364e-9
x_1328 = -9.818181819820193e-9
x_1329 = -9.870129870967074e-9
x_1330 = -9.848484849058364e-9
x_1331 = -9.818181819820193e-9
x_1332 = -9.886363636686314e-9
x_1333 = -9.870129870967074e-9
x_1334 = -9.848484849058364e-9
x_1335 = -9.818181819820193e-9
x_1336 = -9.886363636686314e-9
x_1337 = -9.69696970150418e-9
x_1338 = -9.870129870967074e-9
x_1339 = -9.870129870967074e-9
x_1340 = -9.848484849058364e-9
x_1341 = -9.870129870967074e-9
x_1342 = -9.848484849058364e-9
x_1343 = -9.818181819820193e-9
x_1344 = -9.870129870967074e-9
x_1345 = -9.848484849058364e-9
x_1346 = -9.818181819820193e-9
x_1347 = -9.886363636686314e-9
x_1348 = -9.870129870967074e-9
x_1349 = -9.848484849058364e-9
x_1350 = 

x_1648 = -9.848484849058364e-9
x_1649 = -9.818181819820193e-9
x_1650 = -9.886363636686314e-9
x_1651 = -9.69696970150418e-9
x_1652 = -9.909090909297453e-9
x_1653 = -9.870129870967074e-9
x_1654 = -9.870129870967074e-9
x_1655 = -9.848484849058364e-9
x_1656 = -9.870129870967074e-9
x_1657 = -9.848484849058364e-9
x_1658 = -9.818181819820193e-9
x_1659 = -9.870129870967074e-9
x_1660 = -9.848484849058364e-9
x_1661 = -9.818181819820193e-9
x_1662 = -9.886363636686314e-9
x_1663 = -9.870129870967074e-9
x_1664 = -9.848484849058364e-9
x_1665 = -9.818181819820193e-9
x_1666 = -9.886363636686314e-9
x_1667 = -9.69696970150418e-9
x_1668 = -9.870129870967074e-9
x_1669 = -9.848484849058364e-9
x_1670 = -9.818181819820193e-9
x_1671 = -9.886363636686314e-9
x_1672 = -9.69696970150418e-9
x_1673 = -9.909090909297453e-9
x_1674 = -9.870129870967074e-9
x_1675 = -9.848484849058364e-9
x_1676 = -9.818181819820193e-9
x_1677 = -9.886363636686314e-9
x_1678 = -9.69696970150418e-9
x_1679 = -9.909090909297453e-9
x_1680 = -9.